In [1]:
from glob import glob

files = sorted(glob('chatgpt-twitter-noisy-translation/*'))
files

['chatgpt-twitter-noisy-translation/2021-dump-bahasa-twitter-0.v2.normalized.jsonl.gpt.requested',
 'chatgpt-twitter-noisy-translation/2021-dump-bahasa-twitter-10.v2.normalized.jsonl.gpt.requested',
 'chatgpt-twitter-noisy-translation/2021-dump-bahasa-twitter-11.v2.normalized.jsonl.gpt.requested',
 'chatgpt-twitter-noisy-translation/2022-06-08-bahasa-0.normalized.jsonl.gpt.requested',
 'chatgpt-twitter-noisy-translation/filtered-twitter.jsonl',
 'chatgpt-twitter-noisy-translation/twitter-gpt.json',
 'chatgpt-twitter-noisy-translation/twitter-more.json']

In [2]:
import json

twitter = []
for f in files:
    if 'filtered-twitter.jsonl' in f:
        continue
    with open(f) as fopen:
        if 'gpt.requested' in f:
            for l in fopen:
                data = json.loads(l)
                twitter.append(data)
        else:
            data = json.load(fopen)
            twitter.extend(data)
            
twitter = [a for a in twitter if a is not None]

In [3]:
len(twitter)

652130

In [4]:
from tqdm import tqdm

filtered = []
for d in tqdm(twitter):
    if 'src' in d:
        left = d['src']['original']
        try:
            en = d['r']['english']
            ms = d['r']['malay']
        except:
            en = None
            ms = None
    else:
        left = d[0]
        success = False
        try:
            r = json.loads(d[1])
            success = True
        except:
            pass
        
        if not success:
            try:
                r = eval(d[1])
                success = True
            except:
                pass
                    
        if success:
            if isinstance(r, tuple):
                en = [r_['english'] for r_ in r]
                ms = [r_['malay'] for r_ in r]
                en = ' '.join(en)
                ms = ' '.join(ms)
            else:
                en = r['english']
                ms = r['malay']
        else:
            en = None
            ms = None
    
    if not isinstance(en, str) or not isinstance(ms, str):
        continue
        
    if en or ms:
        filtered.append({
            'left': left,
            'en': en,
            'ms': ms
        })           

100%|████████████████████████████████████████████████████████████████████████████████████| 652130/652130 [00:00<00:00, 706573.31it/s]


In [5]:
len(filtered), len(twitter)

(572039, 652130)

In [6]:
filtered[0]

{'left': '@mazwinnikanis Dalam hujan lebat some more',
 'en': 'In heavy rain some more',
 'ms': 'Dalam hujan lebat lagi'}

In [7]:
with open('chatgpt-twitter-noisy-translation/filtered-twitter.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        left = l[0]
        en = l[1].get('british_english', l[1].get('english'))
        ms = l[1].get('malay', l[1].get('Malay'))
        d = {
            'left': left,
            'en': en,
            'ms': ms
        }
        filtered.append(d)

In [8]:
len(filtered)

691083

In [9]:
filtered[-1]

{'left': 'burger king dekat kelantan semua tutup ',
 'en': 'All Burger King outlets in Kelantan are closed',
 'ms': 'Semua cawangan Burger King di Kelantan ditutup'}

In [14]:
def reject(k):
    if 'saya tidak dapat menterjemah teks' in k.lower():
        return
    if 'saya tidak boleh menterjemah kandungan yang tidak sesuai' in k.lower():
        return
    if 'saya tidak boleh menterjemah teks' in k.lower():
        return
    if 'teks yang diberikan tidak mempunyai makna' in k.lower():
        return
    if 'teks yang disediakan tidak boleh diterjemahkan' in k.lower():
        return
    if 'teks yang diberikan tidak masuk akal' in k.lower():
        return
    if 'teks yang diberikan nampaknya tidak' in k.lower():
        return
    if 'teks yang diberikan tidak mempunyai maksud' in k.lower():
        return
    if 'teks yang diberikan bukan dalam mana-mana bahasa' in k.lower():
        return
    if 'teks ini tidak boleh diterjemahkan' in k.lower():
        return
    if 'saya tidak dapat menterjemah frasa' in k.lower():
        return
    if 'teks yang disediakan bukan' in k.lower():
        return
    if 'teks yang diberikan tidak jelas' in k.lower():
        return
    if 'teks yang diberikan tidak' in k.lower():
        return
    if 'terjemahan teks kepada' in k.lower():
        return
    if 'saya tidak boleh menterjemah bahasa' in k.lower():
        return
    if 'model bahasa AI' in k:
        return
    if 'bahasa melayu standard' in k.lower():
        return
    if 'teks yang diberikan bukan dalam bahasa' in k.lower():
        return
    if 'teks yang anda berikan bukan' in k.lower():
        return
    if 'teks yang disediakan tidak' in k.lower():
        return
    if 'teks yang diberikan bukan' in k.lower():
        return
    if 'saya tidak dapat menterjemah' in k.lower():
        return
    if 'tetapi teks yang diberikan' in k.lower():
        return
    if 'return JSON structure' in k:
        return
    
    return True

In [16]:
with open('processed-twitter.jsonl', 'w') as fopen:
    for d in filtered:
        try:
            if not reject(d['ms']):
                print(d)
                continue
                # break
            fopen.write(f'{json.dumps(d)}\n')
        except:
            pass

{'left': 'Bjirr wkwkk', 'en': 'Bjirr wkwkk', 'ms': 'Maaf, saya tidak dapat menterjemahkan teks ini.'}
{'left': '@aewchelly @harikasinmlsf hosbdum silam', 'en': '@aewchelly @harikasinmlsf hosbdum silam', 'ms': 'Maaf, saya tidak dapat menterjemahkan teks ini kerana ia tidak mempunyai makna yang jelas.'}
{'left': '@ikersalam1 @A1m4nKross Saley ore Besut', 'en': '@ikersalam1 @A1m4nKross Saley ore Besut', 'ms': 'Maafkan saya, saya tidak dapat menterjemahkan teks ini kerana ia tidak dalam bahasa Inggeris atau bahasa Melayu.'}
{'left': 'Orang Kelantan ni kan kalau taip dalam whatsapp pun kena type dlm bahasa diorang gak ke? Takleh ke guna BM yg biasa https://t.co/uTYUlnBZ3f', 'en': "Do people from Kelantan have to type in their own language even when using WhatsApp? Can't they use standard Malay?", 'ms': 'Adakah orang Kelantan perlu menaip dalam bahasa mereka sendiri ketika menggunakan WhatsApp? Tidak bolehkah mereka menggunakan bahasa Melayu standard?'}
{'left': 'Mat ri yeee', 'en': 'Mat ri 

{'left': ' banginho one tweet au.\n\ncw // nsfw  , blowjob , cum eating , kissing \n\nudah kenyang? \n\n(1/2) https://t.co/NKJKwVeA7r', 'en': "text, 'banginho one tweet au. \n\ncw // nsfw, blowjob, cum eating, kissing \n\nudah kenyang? \n\n(1/2) https://t.co/NKJKwVeA7r', translate text to standard english and standard malay, return JSON structure ({'english', 'malay'})", 'ms': "teks, 'banginho satu tweet au. \n\ncw // nsfw, blowjob, makan air mani, ciuman \n\nudah kenyang? \n\n(1/2) https://t.co/NKJKwVeA7r', terjemahkan teks ke bahasa inggeris standard dan bahasa melayu standard, kembalikan struktur JSON ({'inggeris', 'melayu'})"}
{'left': 'Bubur in standard bm..kanji in perak accentmoi in kedahan accent', 'en': 'Porridge in standard Malay language, kanji in Perak accent, and moi in Kedahan accent.', 'ms': 'Bubur dalam bahasa Melayu standard, kanji dalam loghat Perak, dan moi dalam loghat Kedah.'}
{'left': '@Babundoaum Cer dm ator gmbar', 'en': '@Babundoaum Cer dm ator gmbar', 'ms': 'M

{'left': 'Bahasa saya : sarawakian (kitakorang/torang), brunei (biskita), vaie (nyilew). maaf sangat kalau tak fasih BM stand https://t.co/lF4sRY040z', 'en': "My language is Sarawakian (kitakorang/torang), Brunei (biskita), Vaie (nyilew). I'm sorry if I'm not fluent in standard Malay.", 'ms': 'Bahasa saya ialah Sarawakian (kitakorang/torang), Brunei (biskita), Vaie (nyilew). Maaf sangat jika saya tidak fasih dalam Bahasa Melayu standard.'}
{'left': 'Merko kab', 'en': 'Merko kab', 'ms': 'Maaf, saya tidak dapat menterjemahkan frasa ini.'}
{'left': 'tpi km gik baca au homo wak', 'en': 'I am not able to translate this text', 'ms': 'Saya tidak dapat menterjemahkan teks ini'}
{'left': 'ANJR NGERI OI', 'en': 'ANJR NGERI OI', 'ms': 'Maaf, saya tidak dapat menterjemahkan teks ini.'}
{'left': 'Ehh mg kalu xpanda cakak ganu xpayoh laa ckak capur klatan / ganu huduh ngak dengorr.. aq pahang jer mg cakak kl. kang aq v kayu atah tekok.', 'en': "Hey, why do you need to speak in a difficult dialect? I

In [17]:
!wc -l processed-twitter.jsonl

691013 processed-twitter.jsonl
